In [ ]:
# load modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
# fun to crop img
def fn_crop_image(img_array, y_start, x_start):
    y_heigth = 80
    x_width = 110
    
    crop_image = img_array[y_start:y_start+y_heigth,x_start:x_start+x_width] # set RGB to 0 to only analyze red chanel
    return crop_image

# fun to resize img
# to do if further dimension reduction is desired
    

In [ ]:
# define data directory
DATADIR = "../data/Originals/"

# import labels
label_df = pd.read_csv("../data/labels_old_camera.csv")
label_df = label_df.rename(columns={"Unnamed: 0": "img"})

# retain only labels / img with meaningful label
label_df = label_df[label_df["wind_force"] != "0"]

# array with img names of labeled img
labeled_img = label_df["img"].to_numpy()

# create numeric wind force variable
label_df["wind_force_num"] = np.where(label_df["wind_force"] == "n", 0,
                             np.where(label_df["wind_force"] == "w", 1,
                             np.where(label_df["wind_force"] == "m", 2,
                             np.where(label_df["wind_force"] == "s", 3, -1))))


In [ ]:
training_data = []

def create_training_data():
    for img in labeled_img:
        path = os.path.join(DATADIR, img)
        img_array = cv2.imread(os.path.join(path))
        img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB) # from BGR to RGB
        new_array = fn_crop_image(img_array, 230, 570)
        wind_label = label_df.loc[label_df["img"] == img, "wind_force_num"] # get label to coresponding img
        wind_label = np.ndarray.item(wind_label.to_numpy()) # convert to single scalar integer
        training_data.append([new_array, wind_label])
    
create_training_data()

In [220]:
train_images = []
train_labels = []

for feature, label in training_data:
    train_images.append(feature)
    train_labels.append(label)

# mutate to np.array
X_train = np.array(train_images).reshape(-1, np.array(train_images).shape[1], np.array(train_images).shape[2], np.array(train_images).shape[3])
y_train = np.array(train_labels)


(286,)

In [ ]:
# Notes

# X_train=train_images[:5000,:].reshape(5000,-1)
# y_train=train_labels[:5000]
# X_test=test_images[:1000,:].reshape(1000,-1)
# y_test=test_labels[:1000]
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)